In [3]:
import os
import re
import mailbox
import pandas as pd
import hashlib
from openai import OpenAI
from email.utils import parsedate_to_datetime
import tiktoken
import numpy as np
import streamlit as st
from datetime import datetime as dt
from datetime import timedelta

OPENAI_API_KEY = os.environ["OPEN_AI_API_KEY"]
TARGET_EMAIL = "pressingon1617@gmail.com" 

TO_KEY = {
    'bend4him@hotmail.com': "Ben Duncan - Best Friend",
    'beth@expository.org': "Beth Pinckney - Mother",
    'sportcrazy572@aol.com': "Gretchen Allie - Friend",
    'graywolf512@gmail.com': "Gray Kasko - Cousin",
    'goldgirl511@hotmail.com': "Sarag Russet - Friend",
    'sarahpashe1391@gmail.com': "Sarah Paskiewicz - Friend",
    'elitesweetie@gmail.com': "Amber Elliott - Cousin",
    'papaya_313@hotmail.com': "Laura Russet - Friend",
    'kcahoon14@yahoo.com': "Kyle Cahoonn - Friend",
    'kayharper09@gmail.com': "Kay Harper - Girlfriend",
    'barlowgrl16@yahoo.com': "Jen - Friend"
}

def extract_email(val):
    """
    Simple regex to pull emails out of to string, NOTE: this is not a full email regex, just manually hacked
    together on a flight without internet
    """

    match = re.findall("[a-z_0-9]+@[a-z]+.[a-z]+", val.lower())
    if match:
        return match[0]
    else:
        return None

def format_context_chunk(x):

    return f"Date Sent {x['time']}\nTo {x['to_person']}who is Thomas{x['to_relationship']}\n{x['text']}"
    

In [4]:
df_emails = pd.read_csv('../data/embedded_emails.csv')
# This has all been moved to prep_data.py
# df_emails.loc[df_emails.text.isnull(), "text"] = ""
# df_emails['time'] = pd.to_datetime(df_emails.time.str[0:-6])
# df_emails['to_email'] = df_emails.to.apply(extract_email)
# df_emails['to_person'] = df_emails.to_email.replace(TO_KEY)
# df_emails['to_relationship'] = df_emails.to_person.str.split("-").str[1]
# df_emails['to_person'] = df_emails.to_person.str.split("-").str[0]
# df_emails['context_chunk'] = df_emails.apply(format_context_chunk, axis=1)
df_emails.head(1)

,text,time,from,to,n_tokens,chunk_id,email_id,embeddings,to_email,to_person,to_relationship,context_chunk
0,Hey man! I was so excited when I found out tha...,2007-11-27 10:30:20,Pressingon <pressingon1617@gmail.com>,"""Ben Duncan"" <BenD4him@hotmail.com>, ""Ben Dunc...",55,18ae719455bb5964dd5ca2e9-1,18ae719455bb5964dd5ca2e9,"[-0.026006309315562248, -0.026906032115221024,...",bend4him@hotmail.com,Ben Duncan,Best Friend,Date Sent 2007-11-27 10:30:20\nTo Ben Duncan w...


In [5]:
df_emails.to_csv('../data/embedded_emails.csv', index=False)

In [109]:
def keyword_search(df, keyword, n=10, first_occurance=False):
    """
    Simple keyword search that returns results based on the number of times the keyword appears
    in the context chunk
    """
    df_sub = df.copy()
    df_sub['keyword_occurances'] = df_sub.text.apply(lambda x: len(re.findall(keyword, x.lower())))
    if first_occurance:
        df_sub = df_sub.loc[df_sub.keyword_occurances>0].sort_values(['time', 'chunk_id'])
    else:
        df_sub = df_sub.sort_values('keyword_occurances', ascending=False)
    text = df_sub.head(n).loc[:, "context_chunk"].to_list()
    return "###\n\n###".join(text)

search_results = keyword_search(df_emails, " kay ", first_occurance=True)
print(search_results)

Date Sent 2006-08-29 23:54:06
To Sarah Paskiewicz who is Thomas Friend
Well if Gary is right Jessie and I might be married by then lol. No I'd never go out with her. I don't know, I mean 8 months ago I didn't know you or kay or Danny, or anyone really. So even when you're homeschooled things can change pretty quickly. lol a few varsity players, all the JV players are just like wowed by me lol.###

###Date Sent 2006-11-12 14:36:29
To Sarah Paskiewicz who is Thomas Friend
know you will too, because you're that good of a friend to me. I really appreciate that about you Sarah. Thank you so much. I understand why you would be worried about me just not liking Kay all of a sudden, I would be lying if I said that that thought had never crossed my mind before. That's the thing that I'm most afraid of too. To be completely honest, I don't think###

###Date Sent 2006-11-14 23:34:23
To Kay Harper who is Thomas Girlfriend
greatest discovery of the century lol. Oh man Kay I'm so excited and ecstatic

In [111]:
# Date Search
date = dt(2006, 8, 29)

def date_search(df, date, n=10):
    """
    Simple search pulling emails closest to a specific date. Searches on or before the given date
    """
    df_sub = df.copy()
    # Add one day so that we get all results from the passed in day
    search_date = date + timedelta(days=1)
    text = df_sub.loc[df_sub.time < search_date].sort_values(['time', 'chunk_id'], ascending=[False, True]).head(n)["context_chunk"].to_list()
    return "###\n\n###".join(text)

results = date_search(df_emails, date)
print(results)

Date Sent 2006-08-29 23:54:06
To Sarah Paskiewicz who is Thomas Friend
Well if Gary is right Jessie and I might be married by then lol. No I'd never go out with her. I don't know, I mean 8 months ago I didn't know you or kay or Danny, or anyone really. So even when you're homeschooled things can change pretty quickly. lol a few varsity players, all the JV players are just like wowed by me lol.###

###Date Sent 2006-08-29 23:54:06
To Sarah Paskiewicz who is Thomas Friend
They're all like hey you're the star runningback lol. I always say. Yea the star runningback of a homeschool football team that plays 8 man football, and no one outside of family and friends has even heard about. lol they don't know who my friends are, they just hear Dave and Preston always saying that I always hang out with girls lol. So it's pretty###

###Date Sent 2006-08-29 23:42:00
To Sarah Paskiewicz who is Thomas Friend
hahahahaha wow our team doesn't have nice butts lol. Elihah has tight pants, I have tight pant